# General Linear Models and Least Squares
A key distinction of *statistical* models is that they contain free parameters that are fit to data. 

eg- I know the stock market will go up over time, but I dont know by how much. Therefore I allow the change in stock market price over time to be a free parameter whose numerical value is determined by data.

Crafting a statistical model can be difficult, but finding the free parameters based on fitting the model to data is a simple matter of linear algebra- we just have to put the pieces together.

In [1]:
import numpy as np
import scipy.linalg as sl

import matplotlib.pyplot as plt

## General Linear Models
A statistical model is a set of equations that relates predictors (called *independent variables*) to observations (called the *dependent variables*). In the model of the stock market, the independent variable is *time* and the dependent variable is *stock market price* (the S&P 500 index).

Here we will focus on general linear models, or GLMs. Regression is a type of GLM.

### Terminology
Statisticians use slightly different terminology. The following table shows the key letters and descriptions for vectors and matrices used in GLM.

| LinAlg | Stats | Description|
|--------|-------|-------------------------------------------|
|$\mathbf{A}x=b$|$\mathbf{X}\beta=y$|General Linear Model(GLM)|
|$\mathbf{A}$|$\mathbf{X}$|Design matrix (columns=independent vars, predictors, regressors)|
|$x$|$\beta$|Regression coefficients or beta parameters|
|$b$|$y$|Dependent variable, outcome measure, data|

### Setting up a GLM
Setting up a GLM involves:
1. defining an equation that relates the predictor variables to the dependent variable
2. mapping the observed data onto the equations,
3. transforming the series of equations into a matrix equation
4. solving that equation.

We will use a simple example. Here is a model that predicts adult height based on weight and on parent's height. The equation looks like:

$y = \beta_0+\beta_1 w+\beta_2 h +\epsilon$

$y$ is the height of the individual, $w$ is their weight, and $h$ is the average of the parents heights. $\epsilon$ is an error term (also called a residual) because we cannot reasonably expect that weight and height perfectly determine an individual's height. All the unknown factors in the height calculation are absorbed by this error term.

The hypothesis is that weight and parents' height are important for an individual's height, but we don't know *how* important each variable is. the $\beta$s terms/coefficients/weights tell us how to combine weight and parent's height to predict an individual's height. In other words, it's a linear combination, where the $\beta$s are weights.

$\beta_0$ is called an *intercept*. The intercept term is a vector of all 1s. Without an intercept term, the best-fit line is forced to pass through the origin.

Now that we have an equation, we need to map the observed data onto it. We will use the following data:

|$y$|$w$|$h$|
|---|---|---|
|175|70|177|
|181|86|190|
|159|63|180|
|165|62|172|

Mapping the observed data onto our statistical model involves replicating the equation four times (corresponding to four observations in our dataset), each time replacing the variables $y,w,h$ with the measured data:

$\begin{array}c175=\beta_0+70\beta+177\beta_2\\181=\beta_0+86\beta_1+190\beta_2\\159=\beta_0+63\beta_1+180\beta_2\\165=\beta_0+62\beta_1+172\beta_2\end{array}$

the $\epsilon$ term is omitted for now; We now translate this system of equations into a matrix equation.

$\begin{bmatrix}1&70&177\\1&86&190\\1&63&180\\1&62&172\end{bmatrix}\begin{bmatrix}\beta_0\\\beta_1\\\beta_2\end{bmatrix}=\begin{bmatrix}175\\181\\159\\165\end{bmatrix}$

We can express this term as $\mathbf{X}\beta=y$

## Solving GLMs
the main idea of this section is to solve for the vector of unknown coefficients $\beta$. Simply left-multiply both sides by the left-inverse of $\mathbf{X}$, the design matrix

The solution is:

$\mathbf{X}\beta=y$<br>
$(\mathbf{X}^T\mathbf{X})^{-1}\mathbf{X}^T\mathbf{X}\beta=(\mathbf{X}^T\mathbf{X})^{-1}\mathbf{X}^Ty$<br>
$\beta=(\mathbf{X}^T\mathbf{X})^{-1}\mathbf{X}^Ty$

The final equation is called the *least squares solution* and is one of the most important mathematical equations in applied linear algebra.

There may be variations:<br>
$b=(\mathbf{H}^T\mathbf{WH}+\lambda\mathbf{L}^T\mathbf{L})^{-1}\mathbf{H}^Tx$

We can see the least squares formula embedded in the above formula.

The least squares solution via the left-inverse can be translated directly into Python code:

In [5]:
X = np.array([ [1,175,70],
              [1,181,86],
              [1,159,63],
              [1,165,62] ])
y = np.array([ [175,181,159,165] ]).T

#Xb=y
#(XtX)-1Xt b = (XtX)-1Xty
X_leftinv = np.linalg.inv(X.T@X)@X.T

beta = X_leftinv @ y
print(beta)

[[-6.82121026e-13]
 [ 1.00000000e+00]
 [-7.63833441e-14]]


## Is the solution Exact?
The equation $\mathbf{X}\beta=y$ is exactly solvable when $y$ is in the column space of the design matrix $\mathbf{X}$. So the question is whether the data vector is guaranteed to be in the column space of the statistical model. There is no such guarantee. In fact, the data vector $y$ is almost never in the column space of $\mathbf{X}$.

If the data is in the column space of the matrix, it means that the model accounts for 100% of the variance in the data. But this almost never happens: real-world data contains noise and sampling, and the models are simplifications that do not account for all of the variability.

The solution to the conundrum is that we modify the GLM equation to allow for a discrepancy between the model-predicted data and the observed data. It can be expressed in several equivalent ways:

$\mathbf{X}\beta=y+\epsilon$<br>
$\mathbf{X}\beta+\epsilon=y$<br>
$\epsilon = \mathbf{X}\beta-y$

The interpretation of the first equation is that $\epsilon$ is a residual or an error term that you add to the data vector so that it fits inside the column space of the design matrix. 

The interpretation of the second equation is that the residual term is an adjustment to the design matrix so that it fits the data perfectly.

Finally, the interpretation of the third equation is that the residual is defined as the difference between the model-predicted data and the observed data.

The point of this section is that the observed data is almost never inside the subspace spanned by the regressors. For this reason its also common to see the GLM expressed as $\mathbf{X}=\beta\hat{y}$ where $\hat{y}=y+\epsilon$.

Therefore the goal of the GLM is to find the linear combination of the regressors that gets as close as possible to the observed data. 

## Geometric Perspective on Least Squares
Consider the column space of the design matrix $C(\mathbf{X})$ is a subspace of $\mathbb{R}^M$. It's typically a very low-dimensional subspace (that is, $N << M$), because statistical models tend to have many more observations than predictors. The dependent variable is a vector $y\in \mathbb{R}^M$. The questions at hand are, is the vector $y$ in the column space of the design matrix, and if not, what coordinate inside the column space of the design matrix is as close as possible to the data vector.

The answer to the first question is - no. The second has already been discussed in Chap2.

Our goal is to find the set of coefficients $\beta$ such that the weighted combination of columns in $\mathbf{X}$ minimizes the distance to data vector $y$. We can call that projection vector $\epsilon$. How do we find the vector $\epsilon$ and the coefficients $\beta$? We can use orthogonal vector projection. 

The key insight is that the shortest distance between $y$ and $\mathbf{X}$ is given by the projection vector $y-\mathbf{X}\beta$ that meets $\mathbf{X}$ at a right angle.

So, we can use matrix-vector multiplication to arrive at:

$\mathbf{X}^T\epsilon=0$<br>
$\mathbf{X}^T(y-\mathbf{X}\beta)=0$<br>
$\mathbf{X}^Ty-\mathbf{X}^T\mathbf{X}\beta = 0$<br>
$\mathbf{X}^T\mathbf{X}\beta = \mathbf{X}^Ty$<br>
$\beta = (\mathbf{X}^T\mathbf{X})^{-1}\mathbf{X}^Ty$

We started by thinking about the GLM as a geometric projection of a data vector onto the column space of a design matrix, applied the principle of orthogonal vector projection, and have re-derived the same left-inverse solution that we got from the algebraic approach.

## Why Does Least Squares work?
The 'squares' here refer to squared errors between the predicted and the observed data. There is an error term for each $i$th predicted data point, which is defined as $\epsilon_i = \mathbf{X}_i\beta-y_i$. 

the errors are all captured in one vector:

$\epsilon= \mathbf{X}\beta - y$

If the model is a good fit for the data, then the errors should be small. Therefore, we can say that the objective of model fitting is to select the elements for $\beta$ to minimize the elements in $\epsilon$. But just *minimizing* the errors would cause the model to predict values towards negative infinity. Thus, instead, we minimize the *squared* errors, which correspond to their geometric squared distance to the observed data $y$, regardless of whether the prediction itself is positive or negative. This is the same thing as minimizing the squared norm of the errors, hence the name "Least Squares".

this leads to this modification:

$\lVert\epsilon\rVert^2=\lVert\mathbf{X}\beta-y\rVert^2$

This can be seen as an optimization problem, as in we want to find the set of coefficients that minimizes the squared errors. this can be expressed as follows:

$\min(\beta)\lVert\mathbf{X}\beta-y\rVert^2$

The solution to this optimization can be found by setting the derivative of the objective to zero and applying some differential calculus and some algebra.

$0 = \frac{d}{d\beta}\lVert\mathbf{X}\beta-y\rVert^2=2\mathbf{X}^T(\mathbf{X}\beta-y)$<br>
$0 = \mathbf{X}^T\mathbf{X}\beta-\mathbf{X}^Ty$<br>
$\mathbf{X}^T\mathbf{X}\beta=\mathbf{X}^Ty$<br>
$\beta=(\mathbf{X}^T\mathbf{X})^{-1}\mathbf{X}^Ty$

Again, we started from a different perspective- minimize the squared distance between the model-predicted values and the observed values- again we reach the same solution to least squares that we reached simply by using linear algebra intuition.

## Least Squares via QR
The left inverse method is theoretically sound, but risks numerical instability. This is partly because it requires computing the matrix inverse, which can be a numerically unstable operation.

But it turns out the matrix $\mathbf{X}^T\mathbf{X}$ itself can introduce difficulty. Multiplying a matrix by its transpose has implications for properties such as as the norm and the condition number. (matrices with high condition numbers can be more numerically unstable when squared).

QR decomposition provides a more stable way to solve the least squares problem.

$\mathbf{X}\beta=y$<br>
$\mathbf{QR}\beta=y$<br>
$\mathbf{R}\beta = \mathbf{Q}^Ty$<br>
$\beta = \mathbf{R}^{-1}\mathbf{Q}^Ty$<br>

These equations are slightly simplified from the actual low-level numerical implementations. For eg, $\mathbf{R}$ is the same shape as $\mathbf{X}$ i.e tall (therefore, non-invertible), although only the first $N$ rows are non-zero. Which means that rows $N+1$ through $M$ do not contribute to the solution. Those rows can be removed from $\mathbf{R}$ and from $\mathbf{Q}^Ty$. 

It's not even necessary to invert $\mathbf{R}$- that matrix is upper-triangular, and therefore the solution can be obtained via back-substitution. It's the same as solving a system of equations via the Gauss-Jordan method: augment the coefficients matrix by the constants, row-reduce to obtain the RREF, and extract the solution from the final column of the augmented matrix.

The conclusion here is that QR decomposition solves the least squares problem without squaring $\mathbf{X}^T\mathbf{X}$ and without explicitly inverting a matrix. The main reisk of numerical instability comes from computing $\mathbf{Q}$, but this is stable when done through Householder reflections.